In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

from closet_module import capture_module as cm
from closet_module import pose_module as pm
from closet_module import synthesis_module as sm
from closet_module import yolo_module as ym
from closet_module import cut_img_module as cim
from closet_module import grab_cut_module as gm

In [5]:
def avatar_path():
    num = input('원하는 아바타를 선택하여 주십시요:')
    i = int(num)
    file = 'C:\\Users\\lky\\DeepLearning\\smart_closet\\img_lib\Avatar\\' + f'Avatar{i}.png'
    if os.path.isfile(file):
        path = 'C:\\Users\\lky\\DeepLearning\\smart_closet\\img_lib\Avatar\\' + f'Avatar{i}.png'
    return path

In [ ]:
#아바타 캡쳐 과정
cm.cap()

In [6]:
#아바타 정하기
avatar_path=avatar_path()
print(a)

원하는 아바타를 선택하여 주십시요:1
C:\Users\lky\DeepLearning\smart_closet\img_lib\Avatar\Avatar1.png


In [2]:
#포즈로 웹캠 캡쳐하기
pm.video_angle()

In [5]:
#옷 detecting 과정
file_path='clothes_avatar_test.png'
choice='long sleeve top'
ym.detect_cloth(file_path,choice)

In [2]:
#grab_cut
grab_cut_path='clothes_avatar_test.png'
gm.grabcut(grab_cut_path)

오른쪽 마우스 버튼을 누르고 영역을 지정한 후 n을 누르세요


In [4]:
#좌표구하는과정
path_avatar='cap_avatar.png'
path_cloth='clothes_avatar_test.png'
avatar_rs_land=pm.check_landmark(path_avatar)
cloth_re_land=pm.check_landmark(path_cloth)

(423, 179)
(432, 174)


In [17]:
a=cloth_re_land[0]-ym.box_x
b=cloth_re_land[1]-ym.box_y

In [ ]:
#이미지 자르는 과정
cut_path='top_contour.png'
cm.cut_img(ym.box_x,ym.box_y,ym.box_w,ym.box_h,cut_path)

In [22]:
#합성할 이미지 경로
path1='cap_avatar.png'
path2='cut_test.png'

In [20]:
#합성좌표
syn_x=avatar_rs_land[0]-a
syn_y=avatar_rs_land[1]-b

In [23]:
#이미지 합성
sm.synthesis(syn_x,syn_y,path1,path2)